# Live face comparison

+ Determine if Mipi or USB Video0
+ Read from camera
+ Find all faces in image
+ Indentify most middle face
+ Crop image 
+ Determin face embeddings
+ compare Embeddings - get min value index
+ Find best image and present

In [5]:
import numpy as np
import cv2
import os
#import time
import math 
#from imutils import face_utils
#import dlib
import pandas as pd
from pathlib import Path
from numpy import expand_dims
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import time

# DEFINE VARIABLES
#landmark_predictor = dlib.shape_predictor('/home/base/Documents/Git/Projekte/Face-celeb-rec/shape_predictor_68_face_landmarks.dat')

face_cascade = cv2.CascadeClassifier(str(Path.cwd() / 'haarcascade_frontalface_alt.xml'))
brt = 90  # value could be + or - for brightness or darkness
cap = cv2.VideoCapture(0)
gray=False
p=35# frame size around detected face
width=height=224 # size of the cropped image. Same as required for network
mitte=np.empty(shape=[0, 0])
mittleres_Gesicht_X=()

resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),
                                pooling='avg')  # pooling: None, avg or max

ret, frame = cap.read() 
framemitte=np.shape(frame)[1]/2

while(True):
# CAPTURE FRAME BY FRAME
    
    ret, frame = cap.read() 
    if gray==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    cv2.imshow('frame', frame) 

    

#DECTECT FACE IN VIDEO CONTINUOUSLY       
    faces_detected = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
    for (x,y,w,h) in faces_detected:
        rechteck=cv2.rectangle(frame, (x-p, y-p+2), (x+w+p, y+h+p+2), (0, 255, 0), 2)  
        rechteck=cv2.rectangle(frame, (x-p, y-p+2), (x+int(np.ceil(height))+p, y+int(np.ceil(height))+p+2), (0, 0, 255), 2)  

        cv2.imshow('frame', rechteck)     

# DETECT KEY INPUT  - ESC OR FIND MOST CENTERED FACE  
    key = cv2.waitKey(1)
    if key == 27: #Esc key
        cap.release()
        cv2.destroyAllWindows()
        break
    if key & 0xFF == ord('s'): 
        if mittleres_Gesicht_X is not ():
            mittleres_Gesicht_X=()
        #faces_detected = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
        if faces_detected != (): # only if the cascader detected a face, otherwise error
            start1 = time()
            for (x,y,w,h) in faces_detected:
                mitte=np.append(mitte,(x+w/2))
                
            mittleres_Gesicht_X = (np.abs(mitte - framemitte)).argmin()
            end1 = time()
            print('detect middel face ', end1-start1)
# DETECT FACE
            start2=time()
            print(faces_detected[mittleres_Gesicht_X])
            (x, y, w, h) = faces_detected[mittleres_Gesicht_X]
            img=frame[y-p+2:y+h+p-2, x-p+2:x+w+p-2] #use only the detected face; crop it +2 to remove frame
            #cv2.destroyAllWindows()            
            cv2.imshow('frame', img)
            end2=time()
            print('detect face ',end2-start2)
# DETECT LANDMARKS 
#             outlines=landmark_predictor(frame,faces_detected[mittleres_Gesicht_X])
#             outlines = face_utils.shape_to_np(outlines)
# CROP IMAGE 
            if img.shape > (width,height):
                start3=time()
                print('size good')
                img_small=cv2.resize(img, (width, height), interpolation=cv2.INTER_NEAREST) #resize the image to desired dimensions e.g., 256x256  
                end3=time()
                print('face crop', end3-start3)
#CREATE FACE EMBEDDINGS
                start4=time()
                # Make images the same as they were trained on in the VGGface2 Model
                pixels = img_small.astype('float32')
                samples = expand_dims(pixels, axis=0)
                # prepare the face for the model, e.g. center pixels
                samples = preprocess_input(samples, version=2)
                EMBEDDINGS = resnet50_features.predict(samples)
                #print('.')
                end4=time()
                print('create face embeddings' , end4-start4)
# READ CELEB EMBEDDINGS AND COMPARE  
                start5=time()
                EMBEDDINGS_Celebs=pd.read_json(Path.cwd() / 'EMBEDDINGS_8k.json')
                # print(len(EMBEDDINGS.File))
                EuDist=[]
                for i in range(len(EMBEDDINGS_Celebs.File)):
                    Celebs=np.array(EMBEDDINGS_Celebs.Embedding[i]) 
                    dist = np.linalg.norm(EMBEDDINGS-Celebs)
                    EuDist.append(np.linalg.norm(EMBEDDINGS-Celebs))
                #print(EuDist)
                #print(np.argmin(EuDist))
                folder_idx= EMBEDDINGS_Celebs.Name[np.argmin(EuDist)]
                image_idx = EMBEDDINGS_Celebs.File[np.argmin(EuDist)] 
                print(' Distance value: ', np.argmin(EuDist), ' | ' , 'Name: ', EMBEDDINGS_Celebs.Name[np.argmin(EuDist)],' | ' ,' Filename: ', EMBEDDINGS_Celebs.File[np.argmin(EuDist)])
                #plt.scatter(range(len(EMBEDDINGS.File)),EuDist)
                end5=time()
                print('find facematch', end5-start5)
# PLOT IMAGES       
                start6=time()
            #vis = np.concatenate((img_small, img2), axis=1)
                path=Path.cwd()
                pfad=str(Path.cwd() / 'Celebs' / str(folder_idx) / str(image_idx))
                Beleb=cv2.imread(pfad)
                
                Beleb=cv2.resize(Beleb, (np.shape(img)[0] ,np.shape(img)[1]), interpolation=cv2.INTER_NEAREST)
                numpy_horizontal = np.hstack((img, Beleb))
                
                cv2.namedWindow('thats_you',cv2.WINDOW_NORMAL)
                cv2.resizeWindow('thats_you',5*width,5*height)
                #cv2.imshow('thats_you', Beleb)
                cv2.imshow('thats_you', numpy_horizontal)
                
                #cv2.imshow('cropped',img)  
# CLEARING ALL VARIANLES AND CONTINUE WITH THE PROGRAM
                total=time()
                print('totaltime ', total-start1)
                cv2.waitKey(0) #hit any key
                faces_detected=None
                mittleres_Gesicht_X=None
                img=None
                img_small=None
                pixels=None
                samples=None
                EMBEDDINGS=None
           
                cv2.destroyWindow('thats_you')
                #cv2.destroyAllWindows()
                end6=time()
                print('print create image', end6-start6)
            else: 
                print('image to small')
                faces_detected=None
                mittleres_Gesicht_X=None
                img=None
                img_small=None
                pixels=None
                samples=None
                EMBEDDINGS=None
                 

            
            if key == 27: #Esc key
                break
#             if key == 13: #Enter key
#                 cap.release()
#                 cv2.destroyAllWindows()

                
        else:
            print('noface detected')
            
        

# When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

# Now detect face and crop



/home/base/anaconda3/envs/celeb/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


detect middel face  0.0001499652862548828
[211  56 170 170]
detect face  0.0006730556488037109
size good
face crop 0.000194549560546875
create face embeddings 0.6247012615203857
 Distance value:  2321  |  Name:  John Cusack  |   Filename:  000006.jpg
find facematch 2.793994665145874
totaltime  3.4457197189331055
print create image 13.680292844772339
detect middel face  6.937980651855469e-05


IndexError: index 2 is out of bounds for axis 0 with size 1

In [1]:
import cv2
cap = cv2.VideoCapture(0)

cap.release()
cv2.destroyAllWindows()

# Now with parallel programming:


In [ ]:
import numpy as np
import cv2
import os
#import time
import math 
#from imutils import face_utils
#import dlib
import pandas as pd
from pathlib import Path
from numpy import expand_dims
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import time

from multiprocessing import Process

# DEFINE VARIABLES
#landmark_predictor = dlib.shape_predictor('/home/base/Documents/Git/Projekte/Face-celeb-rec/shape_predictor_68_face_landmarks.dat')

face_cascade = cv2.CascadeClassifier(str(Path.cwd() / 'haarcascade_frontalface_alt.xml'))
brt = 90  # value could be + or - for brightness or darkness
cap = cv2.VideoCapture(0)
gray=False
p=35# frame size around detected face
width=height=224 # size of the cropped image. Same as required for network
mitte=np.empty(shape=[0, 0])
mittleres_Gesicht_X=()

resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),
                                pooling='avg')  # pooling: None, avg or max

ret, frame = cap.read() 
framemitte=np.shape(frame)[1]/2

while(True):
# CAPTURE FRAME BY FRAME
    
    ret, frame = cap.read() 
    if gray==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    cv2.imshow('frame', frame) 

    

#DECTECT FACE IN VIDEO CONTINUOUSLY       
    faces_detected = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
    for (x,y,w,h) in faces_detected:
        rechteck=cv2.rectangle(frame, (x-p, y-p+2), (x+w+p, y+h+p+2), (0, 255, 0), 2)  
        rechteck=cv2.rectangle(frame, (x-p, y-p+2), (x+int(np.ceil(height))+p, y+int(np.ceil(height))+p+2), (0, 0, 255), 2)  

        cv2.imshow('frame', rechteck)     

# DETECT KEY INPUT  - ESC OR FIND MOST CENTERED FACE  
    key = cv2.waitKey(1)
    if key == 27: #Esc key
        cap.release()
        cv2.destroyAllWindows()
        break
    if key & 0xFF == ord('s'): 
        if mittleres_Gesicht_X is not ():
            mittleres_Gesicht_X=()
        #faces_detected = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
        if faces_detected != (): # only if the cascader detected a face, otherwise error
            start1 = time()
            for (x,y,w,h) in faces_detected:
                mitte=np.append(mitte,(x+w/2))
                
            mittleres_Gesicht_X = (np.abs(mitte - framemitte)).argmin()
            end1 = time()
            print('detect middel face ', end1-start1)
# DETECT FACE
            start2=time()
            print(faces_detected[mittleres_Gesicht_X])
            (x, y, w, h) = faces_detected[mittleres_Gesicht_X]
            img=frame[y-p+2:y+h+p-2, x-p+2:x+w+p-2] #use only the detected face; crop it +2 to remove frame
            #cv2.destroyAllWindows()            
            cv2.imshow('frame', img)
            end2=time()
            print('detect face ',end2-start2)
# DETECT LANDMARKS 
#             outlines=landmark_predictor(frame,faces_detected[mittleres_Gesicht_X])
#             outlines = face_utils.shape_to_np(outlines)
# CROP IMAGE 
            if img.shape > (width,height):
                start3=time()
                print('size good')
                img_small=cv2.resize(img, (width, height), interpolation=cv2.INTER_NEAREST) #resize the image to desired dimensions e.g., 256x256  
                end3=time()
                print('face crop', end3-start3)
#CREATE FACE EMBEDDINGS
                start4=time()
                # Make images the same as they were trained on in the VGGface2 Model
                pixels = img_small.astype('float32')
                samples = expand_dims(pixels, axis=0)
                # prepare the face for the model, e.g. center pixels
                samples = preprocess_input(samples, version=2)
                EMBEDDINGS = resnet50_features.predict(samples)
                #print('.')
                end4=time()
                print('create face embeddings' , end4-start4)
# READ CELEB EMBEDDINGS AND COMPARE  
                start5=time()
                EMBEDDINGS_Celebs=pd.read_json(Path.cwd() / 'EMBEDDINGS_8k.json')
                # print(len(EMBEDDINGS.File))
                np.array_split(EMBEDDINGS_Celebs, 4)
                EuDist=[]
                def compareToCeleb(EMBEDDINGS_Celebs, EuDist):
                    for i in range(len(EMBEDDINGS_Celebs.File)):
                        Celebs=np.array(EMBEDDINGS_Celebs.Embedding[i]) 
                        dist = np.linalg.norm(EMBEDDINGS-Celebs)
                        EuDist.append(np.linalg.norm(EMBEDDINGS-Celebs))
                        folder_idx= EMBEDDINGS_Celebs.Name[np.argmin(EuDist)]
                        image_idx = EMBEDDINGS_Celebs.File[np.argmin(EuDist)] 
                        return folder_idx, image_idx,EuDist
                
                
                print(' Distance value: ', np.argmin(EuDist), ' | ' , 'Name: ', EMBEDDINGS_Celebs.Name[np.argmin(EuDist)],' | ' ,' Filename: ', EMBEDDINGS_Celebs.File[np.argmin(EuDist)])
                #plt.scatter(range(len(EMBEDDINGS.File)),EuDist)
                end5=time()
                print('find facematch', end5-start5)
# PLOT IMAGES       
                start6=time()
            #vis = np.concatenate((img_small, img2), axis=1)
                path=Path.cwd()
                pfad=str(Path.cwd() / 'Celebs' / str(folder_idx) / str(image_idx))
                Beleb=cv2.imread(pfad)
                
                Beleb=cv2.resize(Beleb, (np.shape(img)[0] ,np.shape(img)[1]), interpolation=cv2.INTER_NEAREST)
                numpy_horizontal = np.hstack((img, Beleb))
                
                cv2.namedWindow('thats_you',cv2.WINDOW_NORMAL)
                cv2.resizeWindow('thats_you',5*width,5*height)
                #cv2.imshow('thats_you', Beleb)
                cv2.imshow('thats_you', numpy_horizontal)
 
                #cv2.imshow('cropped',img)  
# CLEARING ALL VARIANLES AND CONTINUE WITH THE PROGRAM
                total=time()
                print('totaltime ', total-start1)
                cv2.waitKey(0) #hit any key
                faces_detected=None
                mittleres_Gesicht_X=None
                img=None
                img_small=None
                pixels=None
                samples=None
                EMBEDDINGS=None
           
                cv2.destroyWindow('thats_you')
                #cv2.destroyAllWindows()
                end6=time()
                print('print create image', end6-start6)
            else: 
                print('image to small')
                faces_detected=None
                mittleres_Gesicht_X=None
                img=None
                img_small=None
                pixels=None
                samples=None
                EMBEDDINGS=None
                 

            
            if key == 27: #Esc key
                break
#             if key == 13: #Enter key
#                 cap.release()
#                 cv2.destroyAllWindows()

                
        else:
            print('noface detected')
            
        

# When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

# Now detect face and crop



In [ ]:

import numpy as np
import cv2
import os
#import time
import math 
#from imutils import face_utils
#import dlib
import pandas as pd
from pathlib import Path
from numpy import expand_dims
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from time import time

from multiprocessing import Process


EMBEDDINGS_Celebs=pd.read_json(Path.cwd() / 'EMBEDDINGS_8k.json')
# print(len(EMBEDDINGS.File))
Wasgeht=np.array_split(EMBEDDINGS_Celebs, 4)

